In [4]:
import pandas as pd
import requests
import json
import ast
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
sns.set()
from sklearn.cluster import KMeans

In [66]:
#read the various .csv files
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [24]:
train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [25]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [70]:
#Convert Sex, Cabin and Embarked into numerals
def cleaned_data(df):

    #Sex, 0=Male, 1 = Female
    df['Gender_Flag'] = df['Sex'].apply(lambda x: 0 if x == 'male' else 1)
    #Fare
    conditions = [
        (df['Cabin'].str[0] == 'A'),
        (df['Cabin'].str[0] == 'B'),
        (df['Cabin'].str[0] == 'C'),
        (df['Cabin'].str[0] == 'D'),
        (df['Cabin'].str[0] == 'E'),
        (df['Cabin'].str[0] == 'F'),
        (df['Cabin'].str[0] == 'G'),
        (df['Cabin'].str[0] == 'H'),
        (df['Cabin'].str[0] == 'I')]
    choices = [1000, 2000, 3000,4000,5000,6000,7000,8000,9000]
    df['Cabin_Floor_Number'] = np.select(conditions, choices, default = 0)
    df['Cabin_Room_Number'] = df['Cabin'].str[1:]
    df['Cabin_Room_Number'] = pd.to_numeric(df['Cabin_Room_Number'],errors='coerce')
    df['Cabin_Room_Number'] = df['Cabin_Room_Number'].replace(np.nan, 0, regex=True)
    df['Cabin_Number'] = df['Cabin_Floor_Number'] + df['Cabin_Room_Number']
    
    
    return df

In [74]:
test = cleaned_data(test)
test.dropna(subset=['Cabin'])

print(test)


     PassengerId  Pclass                                          Name  \
0            892       3                              Kelly, Mr. James   
1            893       3              Wilkes, Mrs. James (Ellen Needs)   
2            894       2                     Myles, Mr. Thomas Francis   
3            895       3                              Wirz, Mr. Albert   
4            896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
..           ...     ...                                           ...   
413         1305       3                            Spector, Mr. Woolf   
414         1306       1                  Oliva y Ocana, Dona. Fermina   
415         1307       3                  Saether, Mr. Simon Sivertsen   
416         1308       3                           Ware, Mr. Frederick   
417         1309       3                      Peter, Master. Michael J   

        Sex   Age  SibSp  Parch              Ticket      Fare Cabin Embarked  \
0      male  34.5      0      0

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
413    NaN
414      C
415    NaN
416    NaN
417    NaN
Name: Cabin, Length: 418, dtype: object